<h1>Capstone Project - The Battle of Neighborhoods</h1>

<h3>Week 5 - Final Report</h3>

<h3>New Shopping Mall Opening in Brooklyn, United States of America</h3>

<b>Step 1: Importing required libraries</b>

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

In [3]:
pip install geopy

     |████████████████████████████████| 112kB 5.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install beautifulsoup4

     |████████████████████████████████| 112kB 6.4MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install geocoder

     |████████████████████████████████| 102kB 6.0MB/s eta 0:00:01
     |████████████████████████████████| 92kB 26.0MB/s eta 0:00:01
     |████████████████████████████████| 829kB 30.7MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [6]:
import urllib.request

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import folium

import requests

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import geocoder # to get coordinates

<b>Step 2: Scrap the data from Wikipedia page</b>

In [43]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Brooklyn").text

In [44]:
soup = BeautifulSoup(data, 'html.parser')

In [45]:
neighborhoodList = []

In [46]:
for row in soup.find_all("div", class_="mw-content-ltr")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [47]:
br_df = pd.DataFrame({"Neighborhood": neighborhoodList})

In [48]:
br_df.head()

,Neighborhood
0,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)"
1,"► Bedford–Stuyvesant, Brooklyn‎ (1 C, 46 P)"
2,"► Bensonhurst, Brooklyn‎ (1 C, 28 P)"
3,"► Boerum Hill‎ (1 C, 15 P)"
4,"► Borough Park, Brooklyn‎ (1 C, 18 P)"


In [49]:
br_df.shape

(120, 1)

<b>Step 3: Retrieve the Geographical Coordinates</b>

In [50]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Brooklyn, New York City, USA'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [51]:
coords = [ get_latlng(neighborhood) for neighborhood in br_df["Neighborhood"].tolist() ]

In [52]:
coords

[[40.679272583512876, -73.9441024467575],
 [40.6785518497517, -73.94416384399554],
 [40.67911440145965, -73.94411562859526],
 [40.67926305324249, -73.94398808351286],
 [40.679272583512876, -73.9441024467575],
 [40.679272583512876, -73.9441024467575],
 [40.67864084702179, -73.9440389284611],
 [40.679272583512876, -73.9441024467575],
 [40.679272583512876, -73.9441024467575],
 [40.67886016894266, -73.94413661798859],
 [40.67926305324249, -73.94398808351286],
 [40.67926305324249, -73.94398808351286],
 [40.679272583512876, -73.9441024467575],
 [40.679272583512876, -73.9441024467575],
 [40.67926305324249, -73.94398808351286],
 [40.678851694052746, -73.94402220697472],
 [40.67926305324249, -73.94398808351286],
 [40.678299152978205, -73.94419057153888],
 [40.67926305324249, -73.94398808351286],
 [40.679272583512876, -73.9441024467575],
 [40.67880883105737, -73.94402538201142],
 [40.67853976831239, -73.94404961947843],
 [40.679272583512876, -73.9441024467575],
 [40.67911440145965, -73.944115628

In [53]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [54]:
br_df['Latitude'] = df_coords['Latitude']
br_df['Longitude'] = df_coords['Longitude']

In [55]:
print(br_df.shape)
br_df

(120, 3)


,Neighborhood,Latitude,Longitude
0,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",40.679273,-73.944102
1,"► Bedford–Stuyvesant, Brooklyn‎ (1 C, 46 P)",40.678552,-73.944164
2,"► Bensonhurst, Brooklyn‎ (1 C, 28 P)",40.679114,-73.944116
3,"► Boerum Hill‎ (1 C, 15 P)",40.679263,-73.943988
4,"► Borough Park, Brooklyn‎ (1 C, 18 P)",40.679273,-73.944102
5,"► Brighton Beach‎ (1 C, 14 P)",40.679273,-73.944102
6,"► Brooklyn Heights‎ (3 C, 39 P)",40.678641,-73.944039
7,► Brooklyn Navy Yard‎ (12 P),40.679273,-73.944102
8,"► Brownsville, Brooklyn‎ (1 C, 12 P)",40.679273,-73.944102
9,"► Bushwick, Brooklyn‎ (1 C, 36 P)",40.678860,-73.944137


In [56]:
#Save the df into a csv file
br_df.to_csv("br_df.csv", index=False)

<b>Step 4: Create a map of Brooklyn</b>

In [57]:
address = 'Brooklyn, USA'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn, New York City, USA {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn, New York City, USA 40.6501038, -73.9495823.


In [58]:
map_br = folium.Map(location=[latitude, longitude], zoom_start=11)

In [59]:
for lat, lng, neighborhood in zip(br_df['Latitude'], br_df['Longitude'], br_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_br)

In [60]:
map_br

In [61]:
# save the map as HTML file
map_br.save('map_br.html')

<b>Step 5: Explore the neighborhoods using Foursquare API</b>

In [62]:
CLIENT_ID = 'LFOBPNP1U5ZF0HFQFVMJPRUIEZIYWBVATLRIPHLELITTKSGZ'
CLIENT_SECRET = 'SYAR4YWJPPUSXV5AELCH1H22FK54KUPCXBB0VOJQ5M4SNSTA'
VERSION = '20180605'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LFOBPNP1U5ZF0HFQFVMJPRUIEZIYWBVATLRIPHLELITTKSGZ
CLIENT_SECRET:SYAR4YWJPPUSXV5AELCH1H22FK54KUPCXBB0VOJQ5M4SNSTA


In [63]:
radius = 10000
LIMIT = 1000

venues = []

for lat, long, neighborhood in zip(br_df['Latitude'], br_df['Longitude'], br_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [64]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(12000, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",40.679273,-73.944102,Bar Lunatico,40.683549,-73.934773,Bar
1,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",40.679273,-73.944102,Covenhoven,40.675143,-73.960203,Beer Bar
2,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",40.679273,-73.944102,Brooklyn Blend,40.692309,-73.945697,Juice Bar
3,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",40.679273,-73.944102,Mekelburg's,40.687571,-73.962370,Gourmet Shop
4,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",40.679273,-73.944102,Brooklyn Museum,40.671521,-73.963677,Art Museum


In [65]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Admiral's Row,100,100,100,100,100,100
"Barren Island, Brooklyn",100,100,100,100,100,100
"Bath Beach, Brooklyn",100,100,100,100,100,100
"Bay Ridge, Brooklyn",100,100,100,100,100,100
"Bedford–Stuyvesant, Brooklyn",100,100,100,100,100,100
"Bensonhurst, Brooklyn",100,100,100,100,100,100
"Bergen Beach, Brooklyn",100,100,100,100,100,100
Beverley Squares,100,100,100,100,100,100
BoCoCa,100,100,100,100,100,100


In [66]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 198 uniques categories.


In [67]:
venues_df['VenueCategory'].unique()

array(['Bar', 'Beer Bar', 'Juice Bar', 'Gourmet Shop', 'Art Museum',
       'Ice Cream Shop', 'Botanical Garden', 'Farmers Market', 'Track',
       'Field', 'Italian Restaurant', 'Bookstore', 'Wine Shop',
       'Israeli Restaurant', 'Building', 'Indie Movie Theater', 'Park',
       'Performing Arts Venue', 'Theater', 'New American Restaurant',
       'Japanese Restaurant', 'Brewery', 'Athletics & Sports',
       'Eastern European Restaurant', 'Movie Theater', 'Cuban Restaurant',
       'Pizza Place', 'Toy / Game Store', 'Comic Shop', 'Cocktail Bar',
       'Pilates Studio', 'Café', 'Cycle Studio', 'Seafood Restaurant',
       'Bakery', 'Yoga Studio', 'Food & Drink Shop', 'Coffee Shop',
       'Concert Hall', 'Moroccan Restaurant', 'Beach', 'Record Shop',
       'Scenic Lookout', 'Soccer Field', 'Pier', 'Bridge', 'Music Venue',
       'Event Space', 'Thai Restaurant', 'Restaurant', 'Salad Place',
       'Art Gallery', 'Hotel', 'Hotel Bar', 'Garden', 'Spa',
       'Furniture / Home Stor

In [69]:
#Check if the results contain "Shopping Plaza", which is our interest
"Shopping Plaza" in venues_df['VenueCategory'].unique()

True

<b>Step 6: Analysis of each neighborhood</b>

In [70]:
# one hot encoding
br_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
br_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [br_onehot.columns[-1]] + list(br_onehot.columns[:-1])
br_onehot = br_onehot[fixed_columns]

print(br_onehot.shape)
br_onehot.head()

(12000, 199)


,Neighborhoods,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bike Trail,Bistro,Bookstore,Botanical Garden,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Club House,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Community Center,Concert Hall,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farmers Market,Field,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Herbs & Spices Store,Historic Site,History Museum,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Market,Mediterranean Restaurant,Memorial Site,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Record Shop,Reservoir,Restaurant,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stables,State / Provincial Park,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tiki Bar,Toy / Game Store,Track,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court,Waterfront,Wine Bar,Wine Shop,Yoga Studio
0,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [71]:
br_grouped = br_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(br_grouped.shape)
br_grouped

(120, 199)


,Neighborhoods,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bike Trail,Bistro,Bookstore,Botanical Garden,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Club House,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Community Center,Concert Hall,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farmers Market,Field,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Herbs & Spices Store,Historic Site,History Museum,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Market,Mediterranean Restaurant,Memorial Site,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Record Shop,Reservoir,Restaurant,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stables,State / Provincial Park,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tiki Bar,Toy / Game Store,Track,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court,Waterfront,Wine Bar,Wine Shop,Yoga Studio
0,Admiral's Row,0.01,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.01,0.08,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.01,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.02,0.00,0.00,0.01,0.03,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.01,0.01,0.02,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.02,0.01,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.01,0.00,0.01,0.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.02,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.13,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.02,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.03,0.03
1,"Barren Island, Brooklyn",0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.01,0.00,0.01,0.00,0.01,0.03,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.03,0.01,0.00,0.00,0.01,0.01,0.00,0.02,0.00

In [72]:
len(br_grouped[br_grouped["Shopping Plaza"] > 0])

11

In [73]:
br_mall = br_grouped[["Neighborhoods","Shopping Plaza"]]

In [74]:
br_mall.head()

,Neighborhoods,Shopping Plaza
0,Admiral's Row,0.0
1,"Barren Island, Brooklyn",0.0
2,"Bath Beach, Brooklyn",0.0
3,"Bay Ridge, Brooklyn",0.0
4,"Bedford–Stuyvesant, Brooklyn",0.0


<b>Step 7: Clustering of Neighborhoods</b>

In [80]:
# set number of clusters
kclusters = 2

br_clustering = br_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(br_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [81]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
br_merged = br_mall.copy()

# add clustering labels
br_merged["Cluster Labels"] = kmeans.labels_

In [83]:
br_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
br_merged.head()

,Neighborhood,Shopping Plaza,Cluster Labels
0,Admiral's Row,0.0,0
1,"Barren Island, Brooklyn",0.0,0
2,"Bath Beach, Brooklyn",0.0,0
3,"Bay Ridge, Brooklyn",0.0,0
4,"Bedford–Stuyvesant, Brooklyn",0.0,0


In [85]:
br_merged = br_merged.join(br_df.set_index("Neighborhood"), on="Neighborhood")

print(br_merged.shape)
br_merged.head() # check the last columns!

(120, 5)


,Neighborhood,Shopping Plaza,Cluster Labels,Latitude,Longitude
0,Admiral's Row,0.0,0,40.829989,-73.971709
1,"Barren Island, Brooklyn",0.0,0,40.683326,-73.985895
2,"Bath Beach, Brooklyn",0.0,0,40.598478,-73.997270
3,"Bay Ridge, Brooklyn",0.0,0,40.621671,-74.009703
4,"Bedford–Stuyvesant, Brooklyn",0.0,0,40.679104,-73.953056


In [86]:
# sort the results by Cluster Labels
print(br_merged.shape)
br_merged.sort_values(["Cluster Labels"], inplace=True)
br_merged

(120, 5)


,Neighborhood,Shopping Plaza,Cluster Labels,Latitude,Longitude
0,Admiral's Row,0.00,0,40.829989,-73.971709
86,"► Brooklyn Heights‎ (3 C, 39 P)",0.00,0,40.678641,-73.944039
85,"► Brighton Beach‎ (1 C, 14 P)",0.00,0,40.679273,-73.944102
84,"► Borough Park, Brooklyn‎ (1 C, 18 P)",0.00,0,40.679273,-73.944102
83,"► Boerum Hill‎ (1 C, 15 P)",0.00,0,40.679263,-73.943988
82,"► Bensonhurst, Brooklyn‎ (1 C, 28 P)",0.00,0,40.679114,-73.944116
81,"► Bedford–Stuyvesant, Brooklyn‎ (1 C, 46 P)",0.00,0,40.678552,-73.944164
80,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",0.00,0,40.679273,-73.944102
79,"Wyckoff Heights, New York",0.00,0,40.703514,-73.917961
78,"Wingate, Brooklyn",0.00,0,40.662194,-73.947877


In [87]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(br_merged['Latitude'], br_merged['Longitude'], br_merged['Neighborhood'], br_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [88]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

<b>Step 8: Having a look at each Cluster Data</b>

<b>Cluster 0:</b>

In [90]:
br_merged.loc[br_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Plaza,Cluster Labels,Latitude,Longitude
0,Admiral's Row,0.0,0,40.829989,-73.971709
86,"► Brooklyn Heights‎ (3 C, 39 P)",0.0,0,40.678641,-73.944039
85,"► Brighton Beach‎ (1 C, 14 P)",0.0,0,40.679273,-73.944102
84,"► Borough Park, Brooklyn‎ (1 C, 18 P)",0.0,0,40.679273,-73.944102
83,"► Boerum Hill‎ (1 C, 15 P)",0.0,0,40.679263,-73.943988
82,"► Bensonhurst, Brooklyn‎ (1 C, 28 P)",0.0,0,40.679114,-73.944116
81,"► Bedford–Stuyvesant, Brooklyn‎ (1 C, 46 P)",0.0,0,40.678552,-73.944164
80,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",0.0,0,40.679273,-73.944102
79,"Wyckoff Heights, New York",0.0,0,40.703514,-73.917961
78,"Wingate, Brooklyn",0.0,0,40.662194,-73.947877


<b>Cluster 2:</b>

In [91]:
br_merged.loc[br_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Plaza,Cluster Labels,Latitude,Longitude
66,South Brooklyn,0.01,1,40.601118,-73.935355
65,"Sheepshead Bay, Brooklyn",0.01,1,40.583580,-73.942480
45,"Homecrest, Brooklyn",0.01,1,40.590480,-73.957660
107,"► Manhattan Beach, Brooklyn‎ (1 C, 9 P)",0.01,1,40.577820,-73.943550
12,Brighton Beach,0.01,1,40.574570,-73.953430
46,"Kensington, Brooklyn",0.01,1,40.575141,-73.982749
19,Chinatowns in Brooklyn,0.01,1,40.586580,-73.945560
39,"Gerritsen Beach, Brooklyn",0.01,1,40.597947,-73.930814
23,Coney Island,0.01,1,40.587997,-73.960409
49,"Manhattan Beach, Brooklyn",0.01,1,40.577820,-73.943550


<b>Observations:</b>

After observation of data, we noticed the following:
1) Most of shopping malls are concentrated on cluster 1, which is in the lower side of Brooklyn.
2) On the other hands, the upper part of Brooklyn has no shopping malls in the neighborhoods.
3) This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls.
4) Meanwhile, shopping malls in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of shopping malls.

Therefore, as an outcome of this project, we recommend property developers to capitalize on these findings to open a new shopping malls in neighborhoods in cluster 0.  